In [1]:
import os
os.environ['KAGGLE_USERNAME'] = "" # username from the json file
os.environ['KAGGLE_KEY'] = "" # key from the json file
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

100% 2.29G/2.29G [00:30<00:00, 27.1MB/s]
100% 2.29G/2.29G [00:30<00:00, 81.4MB/s]


In [ ]:
!unzip "/content/chest-xray-pneumonia" -d "/content"

In [3]:
import numpy as np
import pandas as pd
import os
import cv2
import random
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

from keras.models import Sequential
from keras.utils import np_utils
from keras.models import Sequential
from keras import optimizers
from keras import backend as K
from keras.layers import Dense, Activation, Flatten, Dense,MaxPool2D, Dropout
from keras.layers import Conv2D, MaxPool2D, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

In [4]:
train_dir = "chest_xray/train"
test_dir = "chest_xray/test"
val_dir = "chest_xray/val"

In [5]:
IMG_SIZE = 224

In [6]:
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D
from keras.applications import VGG16

vgg_model = VGG16(input_shape= (IMG_SIZE, IMG_SIZE,3), 
                  weights='imagenet',
                  include_top=False)

for layer in vgg_model.layers:
    layer.trainable = False

58892288/58889256 [==============================] - 1s 0us/step


In [7]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten, BatchNormalization

vgg_output = vgg_model.output

flatten = Flatten()(vgg_output)
Dense_layer1 = Dense(units = 256,activation='relu')(flatten)
Dense_layer2 = Dense(units=128,activation='relu')(Dense_layer1)
Dense_layer3 = Dense(units = 64,activation='relu')(Dense_layer2)
output_layer = Dense(2,activation='sigmoid')(Dense_layer3)

In [8]:
from keras.models import *

model = Model(inputs = vgg_model.input, outputs = output_layer)

In [9]:
import keras
model.compile(optimizer=keras.optimizers.Adam(0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [10]:
from keras.preprocessing.image import ImageDataGenerator

image_generator_train = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.1,
    horizontal_flip = True,
    vertical_flip = False
)


image_generator_test = ImageDataGenerator(
     rescale = 1./255

)

train = image_generator_train.flow_from_directory(train_dir, 
                                            batch_size=32,
                                            class_mode='categorical',
                                            target_size = (IMG_SIZE,IMG_SIZE))

validation = image_generator_test.flow_from_directory(test_dir, 
                                                batch_size=32,
                                                shuffle = True,
                                                class_mode='categorical',
                                                target_size = (IMG_SIZE,IMG_SIZE))

from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau

callback_list = ModelCheckpoint('vgg16_model_file.h5', 
                                 monitor='val_accuracy', 
                                 verbose=True, 
                                 save_best_only=True)

history = model.fit(
    train,
    epochs = 15,
    validation_data = validation,
    validation_steps = len(validation),
    steps_per_epoch = 100,
    callbacks = [callback_list]
)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Epoch 1/15
100/100 [==============================] - 108s 702ms/step - loss: 0.3533 - accuracy: 0.8279 - val_loss: 0.2723 - val_accuracy: 0.8974

Epoch 00001: val_accuracy improved from -inf to 0.89744, saving model to vgg16_model_file.h5
Epoch 2/15
100/100 [==============================] - 67s 671ms/step - loss: 0.1285 - accuracy: 0.9476 - val_loss: 0.2820 - val_accuracy: 0.8958

Epoch 00002: val_accuracy did not improve from 0.89744
Epoch 3/15
100/100 [==============================] - 67s 674ms/step - loss: 0.1468 - accuracy: 0.9350 - val_loss: 0.5049 - val_accuracy: 0.8157

Epoch 00003: val_accuracy did not improve from 0.89744
Epoch 4/15
100/100 [==============================] - 67s 673ms/step - loss: 0.1667 - accuracy: 0.9307 - val_loss: 0.2563 - val_accuracy: 0.9103

Epoch 00004: val_accuracy improved from 0.89744 to 0.91026, saving model to vgg16_model_file.h5
Epoch 5/15
100/100 [=============

In [11]:
from keras.preprocessing import image

img_width, img_height = 224, 224
img = image.load_img('/content/chest_xray/test/PNEUMONIA/person100_bacteria_475.jpeg', target_size = (img_width, img_height))
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)
np.argmax(model.predict(img), axis=1)

array([1])